# Creating density input files

Imports

In [ ]:
import time

t_begin = time.time()

from pathlib import Path
import numpy as np
from astropy import units as u
import importlib
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import datetime
from scipy import integrate, interpolate

from seyfert.utils import general_utils as gu, filesystem_utils as fsu
from seyfert.cosmology.redshift_density import RedshiftDensity
from seyfert.numeric import general as nug


plt.style.use("plt_params.mplstyle")

## Model description

The theoretical redshift density per unit solid angle for a generic probe $A$ is denoted as

$$
\frac{\mathrm{d} N^A}{\mathrm{d} z \mathrm{d} \Omega}(z)
$$

In `SEYFERT` we use the convention that **the solid angle is measured in steradians**. So the above function will have units of inverse steradians $\mathrm{sr}^{-1}$.
The observed redshift density is obtained from this theoretical density by "convolving" it with an instrument response of the form

\begin{equation}
p_{A}(z_{\rm{p}} | z) = 
\frac{1-f_{\rm out}}{\sqrt{2 \pi}\sigma_{\rm{b}}(1+z)} 
\exp \left\{-\frac{1}{2}\left[\frac{z-c_{\rm{b}} z_{\rm{p}}-z_{\rm{b}}}{\sigma_{\rm{b}}(1+z)}\right]^{2}\right\} + 
\frac{f_{\rm out}}{\sqrt{2 \pi} \sigma_{\rm{o}}(1+z)}\exp \left\{-\frac{1}{2}\left[\frac{z-c_{\rm{o}} z_{\rm{p}}-z_{0}}{\sigma_{\rm{o}}(1+z)}\right]^{2}\right\}
\end{equation}

This is the same model adopted in https://arxiv.org/abs/1910.09273. See it for more details.

Assuming to have defined the tomographic binning for probe $A$, the observed normalized densities of probe $A$ at bin $i$ are obtained by

\begin{equation}
n^A_{i}(z) = \frac{\int_{z_{i}^{-}}^{z_{i}^{+}} \mathrm{d} z_{\rm{p}} \, \frac{\mathrm{d} N^A}{\mathrm{d} z \mathrm{d}\Omega}(z) \, p_{A}(z_{\rm{p}} | z)}{\int_{z_{\min }}^{z_{\max }} \mathrm{d} z \int_{z_{i}^{-}}^{z_{i}^{+}} \mathrm{d} z_{\rm{p}} \, \frac{\mathrm{d} N^A}{\mathrm{d} z \mathrm{d}\Omega}(z) \, p_{A}(z_{\rm{p}} | z)}
\end{equation}



## Data to be specified

To correctly create a redshift density file the following data are always needed:

* **`probe`**: must be one of `PhotometricGalaxy`, `SpectroscopicGalaxy` or `Void`;
* **`has_niz_from_input`**: boolean flag to indicate if the convolution with instrument response has to be done or not.
* **`catalog_f_sky`**: the sky fraction covered by the catalog;
* **`z_bin_edges`**: the edges of the tomographic bins to use for the given probe
* **`input_z_domain`**: a numpy array storing the redshift grid on which the input functions are sampled.

Now there are two possibilities: specify the theoretical redshift density $\frac{\mathrm{d} N^A}{\mathrm{d} z \mathrm{d} \Omega}(z)$ as input (`has_niz_from_input = False`), or directly give the observed normalized densities $n_i^A(z)$ at each bin (`has_niz_from_input = True`). In both cases the functions must be sampled on the `input_z_domain` grid described above.

### True redshift density as input

In this case (`has_niz_from_input = False`) two additional inputs must be given to the code:

* **`input_dN_dz_dOmega`**: a numpy array storing the values of the theoretical redshift density per unit solid angle $\frac{\mathrm{d} N^A}{\mathrm{d} z \mathrm{d} \Omega}(z)$, sampled on the `input_z_domain` grid;
* **`instrument_response`**: a dictionary storing the parameters of the above defined instrument response.


### Observed redshift densities as input
In this case (`has_niz_from_input = True`) only one additional input is needed:

* **`input_n_iz`**: numpy array storing the values of the normalized observed redshift densities $n_i^A(z)$, sampled on the `input_z_domain` grid. This must be a 2-D numpy array: the first axis must have the same size as the number of specified tomographic bins, the second axis must have the same size of the `input_z_domain` parameter.

## Convenience functions and variables

In [ ]:
def build_redshift_density(data):
    d = RedshiftDensity()
    for key, value in data.items():
        if key not in d.__dict__:
            raise KeyError(f"Invalid attribute name {key}")
        setattr(d, key, value)
    
    return d


euclid_f_sky = (15000*u.deg**2 / (4*np.pi*u.steradian).to(u.deg**2)).value
photo_z_bins_edges = np.array([0.001, 0.418, 0.560, 0.678, 0.789, 0.900, 1.019, 1.155, 1.324, 1.576, 2.500])
spectro_z_bins_edges = np.array([0.9, 1.1, 1.3, 1.5, 1.8])

photo_z_bins_centers = (photo_z_bins_edges[:-1] + photo_z_bins_edges[1:])/2
spectro_z_bins_centers = (spectro_z_bins_edges[:-1] + spectro_z_bins_edges[1:])/2


## GCph Euclid Redbook density

Here we describe as an example the steps needed to build redshift density files for the photometric galaxy clustering $\rm GC_{ph}$ for Euclid.

In [ ]:
N_z_sampling = 10000

data = {
    "probe" : 'PhotometricGalaxy',
    "has_niz_from_input" : False,
    "instrument_response" : {
        'f_out': 0.1,
        'z_o': 0.1,
        'z_b': 0.0,
        'sigma_o': 0.050,
        'sigma_b': 0.050,
        'c_o': 1.0,
        'c_b': 1.0
    },
    "catalog_f_sky" : euclid_f_sky,
    "z_bin_edges" : photo_z_bins_edges,
    "input_z_domain" : np.linspace(0.001, 2.5, N_z_sampling)
}

### True redshift distribution as input

We assume $30\; \mathrm{galaxies} / \mathrm{arcmin}^2$ as normalization

In [ ]:
n_gals_per_steradian = (30 * u.arcmin**-2).to(u.steradian**-1).value

z0 = 0.9/np.sqrt(2)

def theGalaxyPhotometricDensity(z):
    return (z/z0)**2 * np.exp(-(z/z0)**1.5)

input_dN_dz_dOmega = theGalaxyPhotometricDensity(data["input_z_domain"])
normalization = n_gals_per_steradian/integrate.simps(y=input_dN_dz_dOmega, x=data["input_z_domain"])
input_dN_dz_dOmega *= normalization

data['input_dN_dz_dOmega'] = input_dN_dz_dOmega

Save to disk

In [ ]:
d = build_redshift_density(data)

outfile = Path("sample_output/gcph_dndz_redbook.h5")

if outfile.exists():
    outfile.unlink()

d.saveToHDF5(outfile)

## GCsp Euclid SPV2 density

Read default GCsp SPV2 data from repository. The path to the default data dir is provided by the function `fsu.default_data_dir()`, which should give the correct path independently of where you cloned the SEYFERT repository.

In [ ]:
data_dir = fsu.default_data_dir()
gcsp_input_file = data_dir / "gcsp/granett_original_data/granett_dNdz_sqrdeg_fine_binning.out"

gcsp_input_data = pd.read_table(gcsp_input_file, skiprows=1, delim_whitespace=True)

dN_dz_dOmega = (gcsp_input_data['model1'].values * u.deg**-2).to(u.steradian**-1)
gcsp_spl = interpolate.InterpolatedUnivariateSpline(x=gcsp_input_data['z'], y=dN_dz_dOmega)

### Data

In [ ]:
N_z_sampling = 4000

data = {
    "probe" : 'SpectroscopicGalaxy',
    "has_niz_from_input" : False,
    "instrument_response" : {
        'f_out': 0.0,
        'z_o': 0.1,
        'z_b': 0.0,
        'sigma_o': 0.050,
        'sigma_b': 0.050,
        'c_o': 1.0,
        'c_b': 1.0
    },
    "catalog_f_sky" : euclid_f_sky,
    "z_bin_edges" : spectro_z_bins_edges,
    "input_z_domain" : np.linspace(0.9, 1.8, N_z_sampling)
}

data['input_dN_dz_dOmega'] = gcsp_spl(data['input_z_domain'])

Save to disk

In [ ]:
d = build_redshift_density(data)

outfile = Path("sample_output/gcsp_dndz_4_bins.h5")

if outfile.exists():
    outfile.unlink()

d.saveToHDF5(outfile)

# Creating bias input files

In [ ]:
from seyfert.cosmology import bias

## GCph piecewise bias

Here we build the piecewise bias file for the photometric galaxy clustering $\rm GC_{ph}$ for Euclid.

In [ ]:
ph_bias = bias.Bias(probe="PhotometricGalaxy")
ph_bias.z_bin_edges = photo_z_bins_edges

ph_bias.model_name = 'piecewise'

bgph_vals = np.sqrt(1 + photo_z_bins_centers)

ph_bias.nuisance_parameters = {f"bgph{i}": bgph_vals[i] for i in range(len(bgph_vals))}
ph_bias.additional_parameters = {}

In [ ]:
outfile = Path("sample_output/gcph_bias_piecewise.h5")

if outfile.exists():
    outfile.unlink()

ph_bias.saveToHDF5(outfile)